In [1]:
from pydantic import BaseModel, Field
from pydantic_ai import Agent
from typing import List
import os

Data structure / BaseModel

In [27]:
class PainPoint(BaseModel):
    description: str = Field(description="What's not working for the user. Be specific about the problem, not the solution.")
    impact: str = Field(description="Cost in time, money, or confidence. Include quotes with numbers when available.")
    quote: str = Field(description="User's exact words")
    
class JobToBeDone(BaseModel):
    functional_job: str = Field(description="What task are they trying to complete?")
    emotional_job: str = Field(description="What they care about and how do they want to feel?")
    context: str = Field(description="When/where does this happen?")
    quote: str

class Workaround(BaseModel):
    what_they_do: str = Field(description="The workaround(s) they've created")
    why_needed: str = Field(description="What problem does the workaround solve?")
    cost: str = Field(description="Time/effort/risks this workaround takes")
    quote: str

class DesiredOutcome(BaseModel):
    outcome: str = Field(description="What do they really want?")
    current_gap: str = Field(description="Why can't they achieve this now?")
    quote: str

class BehavioralSignal(BaseModel):
    observation: str = Field(description="What implicit things did they say/do that was revealing?")
    what_it_reveals: str = Field(description="The underlying need or belief")
    quote: str

class InterviewInsights(BaseModel):
    pain_points: List[PainPoint]
    jobs_to_be_done: List[JobToBeDone]
    workarounds: List[Workaround]
    desired_outcomes: List[DesiredOutcome]
    behavioral_signals: List[BehavioralSignal]

Create AI agent that extracts insights:

In [28]:
multi_insight_agent = Agent(
    model='openai:gpt-4o-mini',
    output_type=InterviewInsights,
    system_prompt="""You are an expert product researcher analyzing user interviews using established frameworks (Jobs-to-be-Done, continuous discovery practices).
    
    Extract ALL insights from the interview:
    - Pain points: Problems causing time waste, costs, uncertainty, frustration
    - Jobs-to-be-done: What they're trying to accomplish (functional + emotional goals)
    - Workarounds: Current hacks/solutions they've created
    - Desired outcomes: What do they really want
    - Behavioral signals: Implicit patterns (what they do that reveals underlying needs)
    
    Always include exact quotes as evidence. Be thorough - one interview may have multiple insights of each type."""
)

Sample Interview:

In [29]:
SAMPLE_INTERVIEW = """
Interviewer: Tell me about the last time you tried to analyze user research data.

Participant: Oh, just last week. We had 12 customer interviews and I needed to present insights to the team. I probably spent 8 hours just going through transcripts, copying quotes into a doc, trying to find themes.

Interviewer: What made it take so long?

Participant: The biggest problem is that insights are scattered everywhere. One user mentions pricing is confusing in passing, another user describes the same issue but uses completely different words. I have to read everything multiple times and manually connect the dots. By the end, my brain is fried and I'm not even confident I caught all the important patterns.

Interviewer: How does that affect your work?

Participant: It makes me anxious honestly. When my PM asks "what did users say about feature X?" I sometimes have to say "let me go back and check" because I can't remember or didn't catch it the first time. It makes me look unprepared even though I've put in the hours.
"""

Extract data models from the sample interview:

In [34]:
result = await multi_insight_agent.run(SAMPLE_INTERVIEW)
insights = result.output

print("\nPAIN POINTS:")
for i, pp in enumerate(insights.pain_points, 1):
    print(f"\n{i}. {pp.description}")
    print(f"   Impact: {pp.impact}")
    print(f"   Quote: {pp.quote}")

print("\n\nJOBS-TO-BE-DONE:")
for i, job in enumerate(insights.jobs_to_be_done, 1):
    print(f"\n{i}. Functional: {job.functional_job}")
    print(f"   Emotional: {job.emotional_job}")
    print(f"   Context: {job.context}")

print("\n\nWORKAROUNDS:")
for i, w in enumerate(insights.workarounds, 1):
    print(f"\n{i}. {w.what_they_do}")
    print(f"   Why: {w.why_needed}")
    print(f"   Cost: {w.cost}")

print("\n\nDESIRED OUTCOMES:")
for i, outcome in enumerate(insights.desired_outcomes, 1):
    print(f"\n{i}. {outcome.outcome}")
    print(f"   Gap: {outcome.current_gap}")

print("\n\nBEHAVIORAL SIGNALS:")
for i, signal in enumerate(insights.behavioral_signals, 1):
    print(f"\n{i}. {signal.observation}")
    print(f"   Reveals: {signal.what_it_reveals}")


PAIN POINTS:

1. Insights are scattered everywhere, making it difficult to see connections.
   Impact: Time wasted in searching for themes and connections.
   Quote: The biggest problem is that insights are scattered everywhere.

2. Reading everything multiple times and still feeling unsure about insights.
   Impact: Uncertainty about missing important patterns leads to anxiety.
   Quote: By the end, my brain is fried and I'm not even confident I caught all the important patterns.

3. Feeling unprepared when asked questions about user feedback.
   Impact: Lack of confidence in presenting findings.
   Quote: I sometimes have to say 'let me go back and check' because I can't remember.


JOBS-TO-BE-DONE:

1. Functional: Analyze user research data and extract key insights.
   Emotional: Feel confident in presenting findings to the team.
   Context: After conducting customer interviews and before a team presentation.


WORKAROUNDS:

1. Manually connect the dots between different user quote